In [ ]:
pip install -U matplotlib pandas triton tabulate

# 低内存丢弃

在本教程中，您将编写一个内存高效的丢弃实现，其状态将由一个单一的 int32 种子组成。这与更传统的丢弃实现不同，后者的状态通常由与输入形状相同的位掩码张量组成。

通过这样做，您将了解：
- 使用 PyTorch 的简单实现丢弃的局限性。
- Triton 中的并行伪随机数生成。

## 基线

在[SRIVASTAVA2014]中首次引入了 dropout 操作符，作为在低数据环境下提高深度神经网络性能的一种方法（即正则化）。

它接受一个向量作为输入，并生成一个形状相同的向量作为输出。输出中的每个标量都有概率 `p` 被更改为零，否则它将从输入中复制。这迫使网络在仅有 `1 - p` 个输入标量可用时仍能表现良好。

在评估时，我们希望利用网络的全部能力，因此我们设置为 `p = 0`。简单来说，这会增加输出的范数（这可能是一个坏事，例如，它可能导致输出 softmax 温度的人工降低）。为了防止这种情况，我们将输出乘以 `1/(1 - p)`，这使得无论 dropout 概率如何，范数保持一致。

让我们首先看看基线实现。

In [ ]:
# 导入 tabulate 库，用于以美观的表格形式打印输出结果。
import tabulate
import torch
import triton
import triton.language as tl

# 动态检测并设置当前可用的 GPU 设备。
DEVICE = triton.runtime.driver.active.get_active_torch_device()


# ---
# 整体概览
# 这是一个为 Dropout 操作编写的、高性能的 GPU 核函数。
# Dropout 是一种在神经网络中常用的正则化技术，它在训练过程中以一定的概率 `p` 将输入张量中的
# 某些元素置为零，以防止过拟合。
# 这个内核实现了“倒置 Dropout (Inverted Dropout)”：对于未被置零的元素，
# 它会除以 `(1 - p)` 进行放大。这样做的好处是，在推理阶段无需对网络权重做任何调整，
# 使得推理过程更高效。
# ---
@triton.jit
def _dropout(
    x_ptr,      # 指向输入张量 `x` 的指针。
    x_keep_ptr, # 指向一个掩码张量 `x_keep` 的指针，其中 1 代表保留，0 代表丢弃。
    output_ptr, # 指向输出张量的指针。
    n_elements, # 输入张量中的元素总数。
    p,          # 一个元素被置为零的概率。
    BLOCK_SIZE: tl.constexpr, # 每个程序实例处理的元素数量，是一个编译期常量。
):
    # --- 并行任务分配 ---
    # 获取当前程序实例的唯一ID。
    pid = tl.program_id(axis=0)
    # 计算当前程序实例需要处理的数据块的起始位置。
    block_start = pid * BLOCK_SIZE
    # 创建当前块内的偏移量向量，即 [0, 1, 2, ..., BLOCK_SIZE-1]。
    offsets = block_start + tl.arange(0, BLOCK_SIZE)
    # 创建一个掩码，防止因 `n_elements` 不是 `BLOCK_SIZE` 的整数倍而导致的越界访问。
    mask = offsets < n_elements

    # --- 数据加载 ---
    # 根据指针和偏移量，安全地从 DRAM 加载输入数据 `x` 和保留掩码 `x_keep` 到 SRAM。
    x = tl.load(x_ptr + offsets, mask=mask)
    x_keep = tl.load(x_keep_ptr + offsets, mask=mask)

    # --- 核心计算：倒置 Dropout ---
    # 这是实现倒置 Dropout 的关键一行。
    # `tl.where(condition, value_if_true, value_if_false)` 是 Triton 中的三元运算符。
    # 如果 `x_keep` 中对应位置的值为 1 (True)，则保留该元素并将其放大 `1 / (1 - p)` 倍。
    # 如果 `x_keep` 中对应位置的值为 0 (False)，则将该元素置为 0.0。
    output = tl.where(x_keep, x / (1 - p), 0.0)

    # --- 数据写回 ---
    # 将计算好的 `output` 向量从 SRAM 安全地写回到 DRAM。
    tl.store(output_ptr + offsets, output, mask=mask)


# ---
# 整体概览
# 这是一个 "主机端 (host-side)" 的封装函数，为用户提供了一个简单的 Python 接口来调用 `_dropout` 内核。
# 它负责处理所有启动 GPU 内核前的准备工作。
# ---
def dropout(x, x_keep, p):
    # 预分配用于存储结果的输出张量。
    output = torch.empty_like(x)
    # 检查输入张量 `x` 是否为内存连续的，这是指针计算正确性的前提。
    assert x.is_contiguous()
    # 获取张量中的元素总数。
    n_elements = x.numel()
    # 定义启动网格（Launch Grid）的大小。
    # `triton.cdiv` 是向上取整除法，确保启动足够多的程序实例来覆盖所有元素。
    grid = lambda meta: (triton.cdiv(n_elements, meta['BLOCK_SIZE']), )
    # 使用计算好的 `grid` 来配置并启动 `_dropout` 内核，并传入所有必要的参数。
    _dropout[grid](x, x_keep, output, n_elements, p, BLOCK_SIZE=1024)
    # 返回计算结果。
    return output


# --- 示例用法与验证 ---

# 创建一个大小为10的随机输入张量，并放置在 GPU 上。
x = torch.randn(size=(10, ), device=DEVICE)

# 设置 dropout 概率 p 为 0.5。
p = 0.5
# 生成 dropout 掩码 `x_keep`。
# `torch.rand(...) > p` 会生成一个布尔类型的掩码，其中大约一半的元素为 True。
# `.to(torch.int32)` 将布尔掩码（True/False）转换为整数（1/0），以便传递给 Triton 内核。
x_keep = (torch.rand(size=(10, ), device=DEVICE) > p).to(torch.int32)

# 调用我们自己编写的 dropout 函数来计算输出。
output = dropout(x, x_keep=x_keep, p=p)

# 使用 `tabulate` 库将输入、保留掩码和最终输出以表格形式打印出来，
# 这样可以非常直观地看到 Dropout 操作的效果。
print(tabulate.tabulate([
    ["input"] + x.tolist(),
    ["keep mask"] + x_keep.tolist(),
    ["output"] + output.tolist(),
]))

## Seeded dropout

上述 dropout 的实现工作良好，但处理起来可能有点尴尬。首先，我们需要存储 dropout 掩码以进行反向传播。其次，当使用重新计算/检查点时，dropout 状态管理可能会变得非常棘手（例如，参见 https://pytorch.org/docs/stable/checkpoint.html 中关于 preserve_rng_state 的所有说明）。在本教程中，我们将描述一种替代实现，它 (1) 具有更小的内存占用；(2) 需要更少的数据移动；并且 (3) 简化了在多次调用内核时保持随机性的管理。

在 Triton 中，伪随机数生成非常简单！在本教程中，我们将使用 triton.language.rand 函数，该函数在给定种子和一组 int32 偏移量的情况下生成一个均匀分布的 float32 值块，范围在[0, 1)之间。但如果您需要，Triton 还提供其他随机数生成策略。

让我们把所有内容整合在一起。

In [ ]:
# ---
# 整体概览
# 这是一个更高级、更高效的 Dropout GPU 核函数。与前一个版本相比，其最大的改进在于它不再需要
# 从外部接收一个预先生成好的掩码张量 `x_keep`。相反，它利用一个 `seed`（种子）和 Triton 内置的
# 伪随机数生成器 `tl.rand`，直接在 GPU 的高速缓存（SRAM）中动态地、确定性地生成 dropout 掩码。
# 这种方法的巨大优势是，它完全消除了在慢速的全局内存（DRAM）中创建和读写一个巨大的掩码张量的
# 需要，极大地节省了内存带宽和开销，是一种典型的通过计算换取内存访问的优化策略。
# ---
@triton.jit
def _seeded_dropout(
    x_ptr,
    output_ptr,
    n_elements,
    p,
    seed, # 新增参数：用于伪随机数生成的种子。
    BLOCK_SIZE: tl.constexpr,
):
    # --- 并行任务分配与地址计算 (与之前版本相同) ---
    pid = tl.program_id(axis=0)
    block_start = pid * BLOCK_SIZE
    offsets = block_start + tl.arange(0, BLOCK_SIZE)
    
    # --- 数据加载 ---
    mask = offsets < n_elements
    x = tl.load(x_ptr + offsets, mask=mask)
    
    # --- 核心计算：动态生成掩码并应用 Dropout ---
    # `tl.rand(seed, offsets)` 是 Triton 的伪随机数生成器。
    # 它为当前块内的每个元素生成一个 [0, 1) 区间内的随机数。
    # 关键点：对于相同的 `seed` 和 `offsets` 组合，生成的随机数序列是完全确定的。
    random = tl.rand(seed, offsets)
    # 直接在 SRAM 中根据生成的随机数创建保留掩码 `x_keep`。
    # 这一步取代了从 DRAM 加载 `x_keep_ptr` 的操作。
    x_keep = random > p
    
    # --- 应用倒置 Dropout 并写回 (与之前版本相同) ---
    output = tl.where(x_keep, x / (1 - p), 0.0)
    tl.store(output_ptr + offsets, output, mask=mask)


# ---
# 整体概览
# 这是 `_seeded_dropout` 内核的主机端封装函数。
# 它接收一个 `seed` 参数而不是一个掩码张量。
# ---
def seeded_dropout(x, p, seed):
    output = torch.empty_like(x)
    assert x.is_contiguous()
    n_elements = x.numel()
    grid = lambda meta: (triton.cdiv(n_elements, meta['BLOCK_SIZE']), )
    # 调用内核时，传入 `seed` 而不是 `x_keep` 指针。
    _seeded_dropout[grid](x, output, n_elements, p, seed, BLOCK_SIZE=1024)
    return output


# --- 示例用法与验证 ---

x = torch.randn(size=(10, ), device=DEVICE)

# 调用 seeded_dropout 函数。注意注释中提到的关键点：dropout 掩码从未在 PyTorch 层面被实例化！
# 第一次调用，使用种子 123。
output = seeded_dropout(x, p=0.5, seed=123)
# 第二次调用，使用完全相同的种子 123。
output2 = seeded_dropout(x, p=0.5, seed=123)
# 第三次调用，使用一个不同的种子 512。
output3 = seeded_dropout(x, p=0.5, seed=512)

# --- 结果打印与分析 ---
# 打印结果表格，用于展示“有籽”随机性的效果。
# 预期结果：
# - "output (seed = 123)" 的两行结果应该是完全一样的，因为相同的种子和输入会生成完全相同的确定性随机掩码。
# - "output (seed = 512)" 这一行的结果应该与前两行不同，因为不同的种子会生成不同的随机掩码。
# 这清晰地证明了该内核的确定性随机行为，以及其在节省内存方面的优势。
print(
    tabulate.tabulate([
        ["input"] + x.tolist(),
        ["output (seed = 123)"] + output.tolist(),
        ["output (seed = 123)"] + output2.tolist(),
        ["output (seed = 512)"] + output3.tolist(),
    ]))

瞧！我们有一个 Triton 内核，它应用相同的 dropout 掩码，只要种子相同！如果您想进一步探索伪随机性在 GPU 编程中的应用，我们鼓励您查看 python/triton/language/random.py！